In [ ]:
ATS_

In [4]:
import qcodes.instrument_drivers.AlazarTech.ATS9440 as ATS_driver
from qcodes.instrument_drivers.AlazarTech import ATS_acquisition_controllers

ATS = ATS_driver.ATS9440('ATS')
ATS_acquisition_controller = ATS_acquisition_controllers.Average_AcquisitionController(name='ATS_control', alazar_id=0)

TypeError: *** error on Instruments ***
while executing query: ('ASK', 'new', (<class 'qcodes.instrument_drivers.AlazarTech.ATS9440.ATS9440'>, 1, 'ATS'), {})

(<bound method InstrumentServer.handle_new of <qcodes.instrument.server.InstrumentServer object at 0x0000000004A475F8>>, (<class 'qcodes.instrument_drivers.AlazarTech.ATS9440.ATS9440'>, 1, 'ATS'), {})
Traceback (most recent call last):
  File "c:\users\lab.blug12meas\documents\qcodes\qcodes\process\server.py", line 347, in _process_ask
    response = func(*args, **kwargs)
  File "c:\users\lab.blug12meas\documents\qcodes\qcodes\instrument\server.py", line 157, in handle_new
    ins = instrument_class(*args, server_name=None, server=self, **kwargs)
TypeError: __init__() got an unexpected keyword argument 'server'


In [ ]:
import os
import clr
import sys
from System import Array
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
# sys.path.append(os.getcwd())
%matplotlib inline

# ATS = ATS9440.ATS9440('ATS',server_name=None)

In [ ]:
ATS.config(trigger_source1='CHANNEL_C',
           trigger_level1=129,
           trigger_engine2='TRIG_ENGINE_K',
           channel_range=[2,2],
           sample_rate=5e6)

result = ATS.acquire(acquisition_controller=ATS_acquisition_controller,
                     buffer_timeout=5000,
                     samples_per_record=1600,
                     records_per_buffer=10,
                     buffers_per_acquisition=2)

In [ ]:
plt.plot(result[0])